# Combining information from two tables 

This notebook contains code to compare and copy-paste information from one dataframe to another if the identity is the same.
This notebook contains:
<br> A) Combining information between tables of presynaptic partners
<br> B) Combining information between tables of presynaptic partners and databse of postsynaptic partners
<br> C) Combining information between tables of postsynaptic partners and databse of postsynaptic partners
<br> D) Combining information between tables for neuron authorship

In [ ]:
import os
import math
import pandas as pd
import numpy as np
from fafbseg import flywire
from caveclient import CAVEclient
client = CAVEclient('flywire_fafb_production')

## A) Combining information between tables of presynaptic partners


### Chossing files of interest and data to transfer

In [ ]:
# Choose path and file

dataPath = r'Z:\Further projects\Heterogeneity across cell types\data\Excels\drive-data-sets' # write your path
fileName_list = ['Mi1_neurons_input_count_R_20240610.xlsx',
                 'Mi4_neurons_input_count_R_20240610.xlsx']

filePath_list = []
for fileName in fileName_list:
    filePath_list.append(os.path.join(dataPath,fileName))


In [ ]:
#Choose column information to compare and to transfer
columns_to_compare = ['presynaptic_ID', 'postsynaptic_ID', 'seg_id']
columns_to_transfer_based_on_pre = ['presynaptic_ID','postsynaptic_ID','symbol','guess','hemisphere','lab', 'author','name','twigs proofread (Y/N)', 
                                    'FlyWire proofread (Y/N)','identified_in', 'lab authorship (Y/N)', 'Extra notes as comments (initials)']
columns_to_transfer_based_on_post = ['presynaptic_ID','postsynaptic_ID', 'optic_lobe_id','column_id','detached_lamina (Y/N)', 'healthy_L3 (Y/N)']


### Loading files as distict dataframes

In [ ]:
# The distinct dataframes will be stored in a dictionary

data_frames = dict()

for i in range(1, len(filePath_list)+1):
    data_frames['df_%02d' % i] = pd.read_excel(filePath_list[i-1])
    #Dropping the fisrt row ('asdf' was added as a walk-around to set that column values as type str)
    if data_frames['df_%02d' % i]["postsynaptic_ID"][0] == 'asdf': 
        data_frames['df_%02d' % i] = data_frames['df_%02d' % i].iloc[1: , :]
        data_frames['df_%02d' % i].reset_index(inplace=True,drop=True)

### Updating column information

In [ ]:
# The distinct columns to update and the updated versions of them will be stored dictionaries

columns_to_update = dict()
for df_name in data_frames.keys():
    curr_df = data_frames[df_name]
    for column_name in columns_to_compare:
        columns_to_update[f'{df_name}_{column_name}'] = curr_df[column_name].copy()

In [ ]:
#Replacing the 'INPUTS PROOFREAD' labelled row of the lists for a well-known marker id

marker_id = '720575940628553731' # VM1 ORN
marker_id_update_df = flywire.update_ids(marker_id, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
marker_id_updated = marker_id_update_df["new_id"][0]
for key,value in columns_to_update.items():
    for i, id in enumerate(value):
        if id == 'INPUTS PROOFREAD':
            value[i] = marker_id_updated # Replacement by the marker
    

In [ ]:
# Updating the different columns. Information is stored in a dictionary
updated_columns = dict()
updated_columns_confidence = dict()
for key, value in columns_to_update.items():
    temp_segmentIDs_df = flywire.update_ids(value.tolist(), stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
    updated_value = temp_segmentIDs_df["new_id"]
    confidence_of_update = temp_segmentIDs_df["confidence"]
    updated_columns[key] = updated_value
    updated_columns_confidence[key] = confidence_of_update
    

### Updating dataframe information

In [ ]:
# Restoring the initial INPUTS PROOFREAD' marker
for key, series in updated_columns.items():
    updated_columns[key] = series.replace(to_replace = int(marker_id), value = 'INPUTS PROOFREAD')
    

In [ ]:
#Updating the different dataframes inplace
for df_name, df_values in data_frames.items():
    for column_name in columns_to_compare:
        data_frames[df_name][column_name] =  updated_columns[f'{df_name}_{column_name}']


### Transfering data from a source data frame into another
#### Defining a function that performs the copy-paste operation

In [ ]:
def update_dataframe(source_df, target_df, reference_column1, reference_column2):
    # Create a dictionary mapping from the reference columns to the source DataFrame
    reference_columns = [reference_column1, reference_column2]
    reference_dict = source_df.groupby(reference_columns).first().reset_index().to_dict(orient='records')
    reference_dict = {(row[reference_column1], row[reference_column2]): row for row in reference_dict}

    # Update the target DataFrame based on the reference columns
    for i, row in target_df.iterrows():
        ref1 = row[reference_column1]
        ref2 = row[reference_column2]
        if (ref1, ref2) in reference_dict:
            source_row = reference_dict[(ref1, ref2)]
            target_df.loc[i] = source_row

    return target_df

### Provide the user-chosen columns and reference column as inputs to the function

In [ ]:
source_cols = columns_to_transfer_based_on_pre 
target_cols = columns_to_transfer_based_on_pre 
reference_column1 = 'presynaptic_ID'
reference_column2 = 'postsynaptic_ID'

### Call the function with the source and target data frames and the provided inputs

In [ ]:
source_df = data_frames['df_01'][source_cols].copy()
target_df = data_frames['df_02'][target_cols].copy()

source_df = source_df.astype(str)
target_df = target_df.astype(str)

result_df = update_dataframe(source_df, target_df,reference_column1, reference_column2)
presynaptic_result_df = result_df.copy()

### Repeating the same process but for postsynaptic_id-based information

In [ ]:
source_cols = columns_to_transfer_based_on_post 
target_cols = columns_to_transfer_based_on_post 
reference_column1 = 'presynaptic_ID'
reference_column2 = 'postsynaptic_ID'

source_df = data_frames['df_01'][source_cols].copy()
target_df = data_frames['df_02'][target_cols].copy()

source_df = source_df.astype(str)
target_df = target_df.astype(str)

result_df = update_dataframe(source_df, target_df,reference_column1, reference_column2)
postsynaptic_result_df = result_df.copy()

### Combining the data frames and saving the data in an excel file

In [ ]:
final_df = pd.concat([presynaptic_result_df,postsynaptic_result_df],axis=1)

In [ ]:
#Saving in a new file

str_final_df = final_df.astype(str)

import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")
fileName_list[0]
file_name = f'{fileName_list[1]}_UPDATED_from_{fileName_list[0]}_{date_str}.xlsx'
savePath = os.path.join(dataPath, file_name)
str_final_df.to_excel(savePath, sheet_name='Data frame update')

##  B) Combining information between tables of presynaptic partners and databse of postsynaptic partners

### Chossing files of interest and data to transfer

In [ ]:
# Choose path and file

dataPath = r'C:\Connectomics-Data\FlyWire\Excels\drive-data-sets' # write your path
fileName_list = [f'Mi1 proofreadings.xlsx',
                 'Mi1_neurons_input_count_R_20240610.xlsx']

filePath_list = []
for fileName in fileName_list:
    filePath_list.append(os.path.join(dataPath,fileName))

In [ ]:
#Choose column information to compare and to transfer
columns_to_compare = ['postsynaptic_ID']

### Loading the distinct data frames

In [ ]:
# The distinct dataframes will be stored in a dictionary

data_frames = dict()

for i in range(1, len(filePath_list)+1):
    data_frames['df_%02d' % i] = pd.read_excel(filePath_list[i-1])
    #Dropping the fisrt row ('asdf' was added as a walk-around to set that column values as type str)
    if data_frames['df_%02d' % i]["seg_id"][0] == 'asdf': 
        data_frames['df_%02d' % i] = data_frames['df_%02d' % i].iloc[1: , :]
        data_frames['df_%02d' % i].reset_index(inplace=True,drop=True)

In [ ]:
# Naming some columns different to match the two dataframes
for df_name in data_frames.keys():
    if 'postsynaptic_ID' not in data_frames[df_name].columns:
        data_frames[df_name]['postsynaptic_ID'] = data_frames[df_name]['seg_id']
        

### Updating column information

In [ ]:
# The distinct columns to update and the updated versions of them will be stored dictionaries
columns_to_update = dict()
for df_name in data_frames.keys():
    curr_df = data_frames[df_name]
    for column_name in columns_to_compare:
        columns_to_update[f'{df_name}_{column_name}'] = curr_df[column_name].copy()

In [ ]:
#Replacing the 'INPUTS PROOFREAD' or NaN labelled row of the lists for a well-known marker id

marker_id = '720575940628553731' # VM1 ORN
marker_id_update_df = flywire.update_ids(marker_id, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
marker_id_updated = marker_id_update_df["new_id"][0]
for key,value in columns_to_update.items():
    for i, id in enumerate(value):
        if id == 'INPUTS PROOFREAD':
            value[i] = marker_id_updated # Replacement by the marker
        elif id == float('nan'):
            value[i] = marker_id_updated # Replacement by the marker
        elif math.isnan(float(id)):
            value[i] = marker_id_updated # Replacement by the marker

In [ ]:
# Updating the different columns. Information is stored in a dictionary
updated_columns = dict()
updated_columns_confidence = dict()
for key, value in columns_to_update.items():
    temp_segmentIDs_df = flywire.update_ids(value.tolist(), stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
    updated_value = temp_segmentIDs_df["new_id"]
    confidence_of_update = temp_segmentIDs_df["confidence"]
    updated_columns[key] = updated_value
    updated_columns_confidence[key] = confidence_of_update

### Updating dataframe information

In [ ]:
# Restoring the initial INPUTS PROOFREAD' marker
for key, series in updated_columns.items():
    updated_columns[key] = series.replace(to_replace = int(marker_id), value = 'INPUTS PROOFREAD')
    

In [ ]:
#Updating the different dataframes inplace
for df_name, df_values in data_frames.items():
    for column_name in columns_to_compare:
        data_frames[df_name][f'Updated_{column_name}'] =  updated_columns[f'{df_name}_{column_name}']

In [ ]:
#The following ids have been updated
query_name = 'df_01'
diff_df = data_frames[query_name].astype(str).loc[data_frames[query_name].astype(str)['Updated_postsynaptic_ID'] != data_frames[query_name].astype(str)['postsynaptic_ID']]
print('Previous:')
print(diff_df['postsynaptic_ID'].unique())
print('Updated:')
print(diff_df['Updated_postsynaptic_ID'].unique())

### Transfering data from a source data frame into another
#### Defining a function that performs the copy-paste operation

In [ ]:
def update_dataframe_single_column(source_df, target_df, reference_column):
    # Create a dictionary mapping from the reference column to the source DataFrame
    reference_dict = source_df.groupby(reference_column).first().reset_index().to_dict(orient='records')
    reference_dict = {row[reference_column]: row for row in reference_dict}

    # Update the target DataFrame based on the reference column
    for i, row in target_df.iterrows():
        ref = row[reference_column]
        if ref in reference_dict:
            source_row = reference_dict[ref]
            target_df.loc[i] = source_row

    return target_df

In [ ]:
# Function inputs
source_cols = ['optic_lobe_id', 'column_id','detached_lamina (Y/N)','healthy_L3 (Y/N)','postsynaptic_ID','Updated_postsynaptic_ID','dorso-ventral']
target_cols = ['optic_lobe_id', 'column_id', 'detached_lamina (Y/N)','healthy_L3 (Y/N)','postsynaptic_ID','Updated_postsynaptic_ID','dorso-ventral']
reference_column = 'postsynaptic_ID'

# Selecting dataframes and resetting index
source_df = data_frames['df_01'][source_cols].copy()
source_df.reset_index(inplace = True, drop = True)
target_df = data_frames['df_02'][target_cols].copy()
target_df.reset_index(inplace = True, drop = True)


source_df = source_df.astype(str)
target_df = target_df.astype(str)

# Running the function and compleating the dataset
result_df = update_dataframe_single_column(source_df, target_df,reference_column)
result_df.head()

### Saving back to excel file

In [ ]:
# Creating string for the date
import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")

# Writting in an existing excel file
from openpyxl import load_workbook
book = load_workbook(filePath_list[1])
writer = pd.ExcelWriter(filePath_list[1], engine = 'openpyxl')
writer.book = book

result_df = result_df.astype(str)
result_df.to_excel(writer, sheet_name='Updated_table_'+date_str) #sorted_df
writer.save()
writer.close()

## C) Combining information between tables of postsynaptic partners and database of postsynaptic partners

In [ ]:
# Choose path and file

dataPath = r'Z:\Further projects\Heterogeneity across cell types\data\Excels\drive-data-sets' # write your path

date = '20230621'
fileName_list = [f'Tm9 proofreadings_{date}.xlsx',
                 f'Tm16_neurons_outputs_count_L_Tm9_{date}.xlsx']

filePath_list = []
for fileName in fileName_list:
    filePath_list.append(os.path.join(dataPath,fileName))

In [ ]:
#Choose column information to compare and to transfer
columns_to_compare = ['postsynaptic_ID']

### Loading the distinct data frames

In [ ]:
# The distinct dataframes will be stored in a dictionary

data_frames = dict()

for i in range(1, len(filePath_list)+1):
    data_frames['df_%02d' % i] = pd.read_excel(filePath_list[i-1])
    #Dropping the fisrt row ('asdf' was added as a walk-around to set that column values as type str)
    if data_frames['df_%02d' % i]["seg_id"][0] == 'asdf': 
        data_frames['df_%02d' % i] = data_frames['df_%02d' % i].iloc[1: , :]
        data_frames['df_%02d' % i].reset_index(inplace=True,drop=True)

In [ ]:
# Naming some columns different to match the two dataframes
for df_name in data_frames.keys():
    if 'postsynaptic_ID' not in data_frames[df_name].columns:
        data_frames[df_name]['postsynaptic_ID'] = data_frames[df_name]['seg_id']
        

In [ ]:
#Choose column information to compare and to transfer
columns_to_compare = ['postsynaptic_ID']

### Updating column information

In [ ]:
# The distinct columns to update and the updated versions of them will be stored dictionaries
columns_to_update = dict()
for df_name in data_frames.keys():
    curr_df = data_frames[df_name]
    for column_name in columns_to_compare:
        columns_to_update[f'{df_name}_{column_name}'] = curr_df[column_name].copy()

In [ ]:
#Replacing the 'INPUTS PROOFREAD' or NaN labelled row of the lists for a well-known marker id

marker_id = '720575940628553731' # VM1 ORN
marker_id_update_df = flywire.update_ids(marker_id, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
marker_id_updated = marker_id_update_df["new_id"][0]
for key,value in columns_to_update.items():
    for i, id in enumerate(value):
        if id == 'INPUTS PROOFREAD':
            value[i] = marker_id_updated # Replacement by the marker
        elif id == float('nan'):
            value[i] = marker_id_updated # Replacement by the marker
        elif math.isnan(float(id)):
            value[i] = marker_id_updated # Replacement by the marker

In [ ]:
# Updating the different columns. Information is stored in a dictionary
updated_columns = dict()
updated_columns_confidence = dict()
for key, value in columns_to_update.items():
    temp_segmentIDs_df = flywire.update_ids(value.tolist(), stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
    updated_value = temp_segmentIDs_df["new_id"]
    confidence_of_update = temp_segmentIDs_df["confidence"]
    updated_columns[key] = updated_value
    updated_columns_confidence[key] = confidence_of_update

### Updating dataframe information

In [ ]:
# Restoring the initial INPUTS PROOFREAD' marker
for key, series in updated_columns.items():
    updated_columns[key] = series.replace(to_replace = int(marker_id), value = 'INPUTS PROOFREAD')
    

In [ ]:
#Updating the different dataframes inplace
for df_name, df_values in data_frames.items():
    for column_name in columns_to_compare:
        data_frames[df_name][f'Updated_{column_name}'] =  updated_columns[f'{df_name}_{column_name}']

In [ ]:
#The following ids have been updated
query_name = 'df_01'
diff_df = data_frames[query_name].astype(str).loc[data_frames[query_name].astype(str)['Updated_postsynaptic_ID'] != data_frames[query_name].astype(str)['postsynaptic_ID']]
print('Previous:')
print(diff_df['postsynaptic_ID'].unique())
print('Updated:')
print(diff_df['Updated_postsynaptic_ID'].unique())

### Transfering data from a source data frame into another
#### Defining a function that performs the copy-paste operation

In [ ]:
def update_dataframe_single_column(source_df, target_df, reference_column):
    # Create a dictionary mapping from the reference column to the source DataFrame
    reference_dict = source_df.groupby(reference_column).first().reset_index().to_dict(orient='records')
    reference_dict = {row[reference_column]: row for row in reference_dict}

    # Update the target DataFrame based on the reference column
    for i, row in target_df.iterrows():
        ref = row[reference_column]
        if ref in reference_dict:
            source_row = reference_dict[ref]
            target_df.loc[i] = source_row

    return target_df

In [ ]:
# Function inputs
source_cols = ['XYZ-ME','XYZ-LO','optic_lobe_id', 'column_id','detached_lamina (Y/N)','healthy_L3 (Y/N)','postsynaptic_ID','Updated_postsynaptic_ID']
target_cols = ['XYZ-ME','XYZ-LO','optic_lobe_id', 'column_id', 'detached_lamina (Y/N)','healthy_L3 (Y/N)','postsynaptic_ID','Updated_postsynaptic_ID']
reference_column = 'postsynaptic_ID'

# Selecting dataframes and resetting index
source_df = data_frames['df_01'][source_cols].copy()
source_df.reset_index(inplace = True, drop = True)
target_df = data_frames['df_02'][target_cols].copy()
target_df.reset_index(inplace = True, drop = True)


source_df = source_df.astype(str)
target_df = target_df.astype(str)

# Running the function and compleating the dataset
result_df = update_dataframe_single_column(source_df, target_df,reference_column)
result_df.head()

### Saving back to excel file

In [ ]:

# Creating string for the date
import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")

# Writting in an existing excel file
from openpyxl import load_workbook
book = load_workbook(filePath_list[1])
writer = pd.ExcelWriter(filePath_list[1], engine = 'openpyxl')
writer.book = book

result_df = result_df.astype(str)
result_df.to_excel(writer, sheet_name='Updated_table_'+date_str) #sorted_df
writer.save()
writer.close()

## D) Combining information between tables for neuron authorship

In [ ]:
# Loading the first data frame

# Choose path and file
PC_disc = 'D'
#dataPath = f'{PC_disc}:\Connectomics-Data\FlyWire\Excels\drive-data-sets\submission_nature'
dataPath = f'{PC_disc}:\FlyWire-Data\FlyWire-Authorship'
date = '20240115'
fileName = f'Segment IDs in publication_{date}.xlsx'

filePath = os.path.join(dataPath,fileName)

#Loading file as DataFrame
df = pd.read_excel(filePath)


#Dropping the fisrt row ('asdf' was added as a walk-around to set that column values as type str)
if df["seg_id"][0] == 'asdf': 
    df = df.iloc[1: , :]
    df.reset_index(inplace=True,drop=True)
    
#Dropping dupllicates
segment_publication_df = df.drop_duplicates(subset=["seg_id"], keep='first')



In [ ]:
# Loading the other dataframe

# Choose path and file
PC_disc = 'D'
#dataPath = f'{PC_disc}:\Connectomics-Data\FlyWire\Excels\drive-data-sets\submission_nature'
dataPath = f'{PC_disc}:\FlyWire-Data\FlyWire-Authorship'
date = '20240115'
fileName = f'Multi-neuron-changelog-{date}.xlsx'
filePath = os.path.join(dataPath,fileName)

#Loading file as DataFrame
df = pd.read_excel(filePath)


#Dropping the fisrt row ('asdf' was added as a walk-around to set that column values as type str)
if df["segment_ID"][0] == 'asdf': 
    df = df.iloc[1: , :]
    df.reset_index(inplace=True,drop=True)
    
change_log_df = df.copy()
#change_log_df['segment_ID'] = change_log_df['segment_ID'].str.strip("'")

In [ ]:
segment_publication_df

In [ ]:
# Adding tte ID column from the fisrt table to the second

# Merge the data frames based on the common column "Updated_seg_ids"
change_log_final = change_log_df.copy()
merged_df = pd.merge(change_log_final, segment_publication_df, left_on='segment_ID', right_on='seg_id', how='left')

# Add a new column "neuron type" to change_log_df using information from the "symbol" column
change_log_final['neuron type'] = merged_df['symbol']

In [ ]:
change_log_final

In [ ]:
#Saving in a new file

import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")
file_name = f'change_log_final_{date_str}.xlsx'
savePath = os.path.join(dataPath, file_name)
change_log_final.to_excel(savePath, sheet_name='User-Lab summary')